In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.colors import to_rgb
import os

In [ ]:
GRAPHS_OUTPUT_DIR = './SRDS-graphs-pngs'
graph_png_save_dir = GRAPHS_OUTPUT_DIR
# make dir if it doesn't exist already
if not os.path.exists(f"{GRAPHS_OUTPUT_DIR}/"):
    os.makedirs(f"{GRAPHS_OUTPUT_DIR}/")
# delete existing files in the directory, if any:
existing_files = os.listdir(f'{GRAPHS_OUTPUT_DIR}/')
for f in existing_files:
    os.remove(f"{GRAPHS_OUTPUT_DIR}/{f}")

In [ ]:
# read csv files into memory
csv_dir = './SRDS-results/'
csv_filenames = os.listdir(csv_dir)
csv_filenames[:] = [x for x in csv_filenames if x != '.DS_Store'] # remove '.DS_Store' if it exists

In [ ]:
# sort csv files figures-wise
figs_list = ['7a', '7b', '7c', '7d', '8a', '8b', '8c', '8d', '9a', '9b', '10a', '10b', '11a', '11b', '11c', '13a', '13b', 'A1a', 'A1b']
fig_wise_csvs = {}
for fig_no in figs_list:
    fig_wise_csvs[fig_no] = []
    for file in csv_filenames:
        if file.split('_')[0] == fig_no:
            fig_wise_csvs[fig_no].append(file)
            

In [ ]:
# set up structures to hold dataframes
all_dfs = {}

two_dfs_figs = ['10a', '10b', '11a', '11b']

df_template = pd.DataFrame(
    {
        'sites': [],
        'configuration': [],
        'reconfiguration': [],
        'hurricane': [],
        'method': [],
        'bucket': [],
        'green': [],
        'orange': [],
        'red': [],
        'yellow': [],
        'blue': [],
        'gray': [],
    }
)

for fig in figs_list:
    all_dfs[fig] = []
    df_tmp = df_template.copy()
    all_dfs[fig].append(df_tmp)
    if fig in two_dfs_figs:
        df_tmp2 = df_template.copy()
        all_dfs[fig].append(df_tmp2)

In [ ]:
# make data frames from the csv files and save into all_dfs dict
for fig in fig_wise_csvs:
    for csv_file in fig_wise_csvs[fig]:
        file_content = pd.read_csv(f"./{csv_dir}/{csv_file}")
    
        prob_g = file_content['green_probability'][0]
        prob_o = file_content['orange_probability'][0]
        prob_r = file_content['red_probability'][0]
        prob_y = file_content['yellow_probability'][0]
        prob_b = file_content['blue_probability'][0]
        prob_x = file_content['grey_probability'][0]

        method = file_content['method'][0]
        bucket = file_content['bucket'][0]

        config = (csv_file.split('_')[1]).split('.')[0]
        if config == '1r6+6+6':
            config = '6+6+6'
            rec = 'H->R->CA'
        elif config == '2r6+6+6':
            config = '6+6+6'
            rec = 'H->CA->R'
        else:
            rec = 'No'
        
        if fig not in two_dfs_figs:
            idx = 0
        else:
            tmp = (csv_file.split('_')[2]).split('.')[0]
            if tmp in ['W', 'J']:
                idx = 0
            elif tmp in ['K', 'M']:
                idx = 1
        
        # sites = static_data[fig]['sites']
        # hurr = static_data[fig]['hurr']
        sites = ''
        hurr = ''

        df_row = [sites, config, rec, hurr, method, bucket, prob_g, prob_o, prob_r, prob_y, prob_b, prob_x]
        this_df = all_dfs[fig][idx]
        this_df.loc[-1] = df_row
        this_df.index = this_df.index + 1
        this_df = this_df.sort_index()

In [ ]:
save_pngs_override = False # if this is true then wont save pngs in all cases

In [ ]:
def _make_graph(df_original, vs_graphs=False, auto_title=True, title='', add_hatches=False, save_graph_name=None, verticle_lines=2, fig_w=None, fig_h=None, relative_width_bar=None, override_x_labels=None, overlapping_labels_case=False):
    df = df_original.copy()
    
    # graph specifications:
    _color_list = ["mediumseagreen", "orange", "red", "yellow", "blue", "lightslategrey"]
    color_list = [to_rgb(i) for i in _color_list]
    # patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
    patterns = {
        to_rgb("mediumseagreen"): ".", 
        to_rgb("orange"): "\\", 
        to_rgb("red"): "/", 
        to_rgb("yellow"): "-", 
        to_rgb("blue"): "+", 
        to_rgb("lightslategrey"): "x",
    }
    patterns_color = None if not add_hatches else 'silver'
    rc_param_xtick = 18
    rc_param_ytick = 18
    rc_param_font_size = 18
    x_tick_font_size = 30
    y_tick_font_size = 34
    y_axis_label_font_size = 34
    bars_value_font_size = 34
    vertical_dividers_font_size = 18
    fig_size_w = 15 if fig_w is None else fig_w
    fig_size_h = 10 if fig_h is None else fig_h
    bar_width = 0.8 if relative_width_bar is None else relative_width_bar # width of bar relative to space it is alloted. e.g 0.8 would add some space between the bar and the next one. 1 would leave no space between bars.
    # pd.set_option('expand_frame_repr', False)
    plt.rc('xtick', labelsize=rc_param_xtick) 
    plt.rc('ytick', labelsize=rc_param_ytick)
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams["font.size"] = str(rc_param_font_size)

    # x-axis adjustment:
    if not vs_graphs:
        df.loc[df["reconfiguration"].str.endswith('H->R->CA'), "configuration"] = "rec. 6+6+6\n(H→R→CA)"
        df.loc[df["reconfiguration"].str.endswith('H->CA->R'), "configuration"] = "rec. 6+6+6\n(H→CA→R)"
        df['configuration'] = pd.Categorical(df['configuration'], ["2", "2+2", "6","6+6", "6+6+6", "rec. 6+6+6\n(H→CA→R)", "rec. 6+6+6\n(H→R→CA)"]) # sort order
        df = df.sort_values("configuration")
    df['configuration'] = df['configuration'].replace(['2+2'], '2-2') # internally 2+2 is used to represent 2-2 in the tool
    df['configuration'] = df['configuration'].replace(['6+6'], '6-6') # internally 6+6 is used to represent 6-6 in the tool

    # graph title/subtitle
    if auto_title:
        graph_title = ''
        graph_title += 'hurricane=' + df['hurricane'][0] + '; '
        graph_title += 'sites=' + df['sites'][0] + '; '
        graph_title += 'method=' + df['method'][0] + '; '
        graph_title += 'bucket=' + df['bucket'][0] + '; '
    else:
        graph_title = title

    # generate graph object (+ temporarily drop every column except colors as they are not directly being used for column values):
    # print(graph_title)
    graph = df.drop(['sites', 'reconfiguration', 'hurricane', 'method', 'bucket'], inplace=False, axis=1).plot.bar(x='configuration', stacked=True, title=graph_title, color=color_list, legend=False, figsize=(fig_size_w, fig_size_h), width=bar_width, rot=0, edgecolor=patterns_color) # rot = degree of rotation for x labels
    
    bar_num = 0 if vs_graphs else None
    total_verticle_lines = verticle_lines
    extra_verticle_buffer = -0.02 if not overlapping_labels_case else -0.055
    for bar in graph.patches:
        if add_hatches:
            bar.set_hatch(patterns[(bar.get_facecolor()[0], bar.get_facecolor()[1], bar.get_facecolor()[2])])
        # This is actual value we'll show.
        value = f'{str(np.round(bar.get_height()*100, 2))}%'
        if(bar.get_height()>0.0):
            graph.annotate(
                value,
                # Put the text in the middle of each bar. get_x +half_width, get_y + half_height.
                (bar.get_x()+bar.get_width()/2,(bar.get_y()+np.round(bar.get_height(),decimals=4)/2)+extra_verticle_buffer),
                size=bars_value_font_size,
                ha='center',
                weight='bold'
            )
            extra_verticle_buffer = extra_verticle_buffer + 0.005 if overlapping_labels_case else -0.02
        if vs_graphs and bar_num <= total_verticle_lines+1:
            bar_num += 1
            if (bar_num % 2) == 0: # after even numbered bar, add a verticle line
                y = 0.0
                while y < 1:
                    graph.annotate(
                        '|', 
                        xy=(bar_num - 0.525, y),
                        fontweight='bold',
                        fontsize=vertical_dividers_font_size,
                        )
                    y += 0.1

    # adjust minor issues with x and y ticks
    # the following would apply newline character to x ticks if any
    if override_x_labels is None:
        x_tick_labels = graph.get_xticklabels()
    else:
        x_tick_labels = override_x_labels
    plt.xticks(range(len(x_tick_labels)), x_tick_labels, fontsize=x_tick_font_size)
    # convert y ticks to percentage values:
    # y_tick_labels = graph.get_yticklabels()
    # y_tick_labels = ['0%', '20%', '40%', '60%', '80%', '100%']
    # plt.yticks(range(len(y_tick_labels)), y_tick_labels)
    graph.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.yticks(fontsize=y_tick_font_size)

    # x and y axis labels
    # plt.xlabel("System Configuration")
    plt.xlabel("")
    plt.ylabel("Probability of Operational State", fontsize=y_axis_label_font_size)
    
    if save_graph_name is not None:
        if save_pngs_override == False:
            plt.savefig(f"{graph_png_save_dir}/{save_graph_name}.png",bbox_inches='tight')

In [ ]:
def make_standard_graph(df_key, exclude=None, auto_title=False, title='', save_graph_name=None, verticle_lines=2, fig_w=None, fig_h=None, relative_width_bar=None, override_x_labels=None, overlapping_labels_case=False):
    # standard graph as in for one specific sites-config (e.g. Honolulu+Waiau+DRFortress) with the possible system configs (2-2, 6-6 etc) for the given method and bucket in the df_key. excludes configs in the param exclude

    df = all_dfs[df_key][0].copy()
    if exclude is not None:
        for c in exclude:
            if c in ['H->CA->R', 'H->R->CA']:
                df.drop(df.loc[df['reconfiguration'].str.fullmatch(c)].index, inplace=True)
            elif c in ['2+2', '6+6+6']:
                df.drop(df.loc[df['configuration'].str.endswith(c)].index, inplace=True)
            else:
                df.drop(df.loc[df['configuration'].str.fullmatch(c)].index, inplace=True)
        df.reset_index(inplace = True)
        df.drop(['index'], inplace=True, axis=1)

    _make_graph(df, auto_title=auto_title, title=title, save_graph_name=save_graph_name, verticle_lines=verticle_lines, fig_w=fig_w, fig_h=fig_h, relative_width_bar=relative_width_bar, override_x_labels=override_x_labels, overlapping_labels_case=overlapping_labels_case)

In [ ]:
def make_vs_graph(key_a, key_b, comparison_side_a, comparison_side_b, exclude=None, auto_title=False, title='', save_graph_name=None, verticle_lines=2):
    dfa =  all_dfs[key_a][0].copy()
    dfb =  all_dfs[key_b][1].copy()

    cols_renamed = {}
    if exclude is not None:
        for c in exclude:
            if c in ['H->CA->R', 'H->R->CA']:
                dfa.drop(dfa.loc[dfa['reconfiguration'].str.fullmatch(c)].index, inplace=True)
                dfb.drop(dfb.loc[dfb['reconfiguration'].str.fullmatch(c)].index, inplace=True)
            elif c in ['6+6']:
                dfa.drop(dfa.loc[(dfa['configuration'].str.fullmatch('6+6') & dfa['reconfiguration'].str.fullmatch('No'))].index, inplace=True)
                dfb.drop(dfb.loc[(dfb['configuration'].str.fullmatch('6+6') & dfb['reconfiguration'].str.fullmatch('No'))].index, inplace=True)
            elif c in ['2+2']:
                dfa.drop(dfa.loc[dfa['configuration'].str.endswith('+2')].index, inplace=True)
                dfb.drop(dfb.loc[dfb['configuration'].str.endswith('+2')].index, inplace=True)
            elif c in ['6+6+6']:
                dfa.drop(dfa.loc[(dfa['configuration'].str.fullmatch('6+6+6') & dfa['reconfiguration'].str.fullmatch('No'))].index, inplace=True)
                dfb.drop(dfb.loc[(dfb['configuration'].str.fullmatch('6+6+6') & dfb['reconfiguration'].str.fullmatch('No'))].index, inplace=True)
            else:
                dfa.drop(dfa.loc[dfa['configuration'].str.fullmatch(c)].index, inplace=True)
                dfb.drop(dfb.loc[dfb['configuration'].str.fullmatch(c)].index, inplace=True)

    if exclude is not None:
        if '2+2' not in exclude:
            dfa.loc[dfa["configuration"].str.endswith('2+2'), "configuration"] = f"2-2\n{comparison_side_a}"
            dfb.loc[dfb["configuration"].str.endswith('2+2'), "configuration"] = f"2-2\n{comparison_side_b}"
            cols_renamed['2+2'] = [f"2-2\n{comparison_side_a}", f"2-2\n{comparison_side_b}"]
        if '6+6+6' not in exclude:
            dfa.loc[dfa["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comparison_side_a}"
            dfb.loc[dfb["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comparison_side_b}"
            cols_renamed['6+6+6'] = [f"6+6+6\n{comparison_side_a}", f"6+6+6\n{comparison_side_b}"]
        if '6+6' not in exclude:
            dfa.loc[dfa["configuration"].str.endswith('6+6'), "configuration"] = f"6-6\n{comparison_side_a}"
            dfb.loc[dfb["configuration"].str.endswith('6+6'), "configuration"] = f"6-6\n{comparison_side_b}"
            cols_renamed['6+6'] = [f"6-6\n{comparison_side_a}", f"6-6\n{comparison_side_b}"]

    vs_df = pd.concat([dfa, dfb])

    vs_df['configuration'] = pd.Categorical(vs_df['configuration'], [f"2-2\n{comparison_side_a}", f"2-2\n{comparison_side_b}", f"6-6\n{comparison_side_a}", f"6-6\n{comparison_side_b}", f"6+6+6\n{comparison_side_a}", f"6+6+6\n{comparison_side_b}"]) # sort order
    vs_df = vs_df.sort_values("configuration")
    vs_df.reset_index(inplace = True)
    vs_df.drop(['index'], inplace=True, axis=1)

    _make_graph(vs_df, vs_graphs=True, auto_title=auto_title, title=title, save_graph_name=save_graph_name, verticle_lines=verticle_lines)

In [ ]:
# fig 7a:
key = '7a'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig7a')

In [ ]:
# fig 7b:
key = '7b'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig7b')

In [ ]:
# fig 7c:
key = '7c'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig7c')

In [ ]:
# fig 7d:
key = '7d'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig7d')

In [ ]:
# fig 8a:
key = '8a'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig8a')

In [ ]:
# fig 8b:
key = '8b'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig8b')

In [ ]:
# fig 8c:
key = '8c'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig8c')

In [ ]:
# fig 8d:
key = '8d'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig8d')

In [ ]:
# fig 9a:
key = '9a'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig9a')

In [ ]:
# fig 9b:
key = '9b'
exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig9b')

In [ ]:
# fig 10a:
key_a = key_b = '10a'

comp_str_a = 'Waiau'
comp_str_b = 'Kahe'

# exclude_configs = ['2', '6', 'H->CA->R', 'H->R->CA']
exclude_configs = ['2', '6', 'H->CA->R', 'H->R->CA', '2+2']

make_vs_graph(key_a, key_b, comp_str_a, comp_str_b, exclude=exclude_configs, save_graph_name='fig10a', verticle_lines=1)

In [ ]:
# fig 10b: 
key_a = key_b = '10b'

comp_str_a = 'Waiau'
comp_str_b = 'Kahe'

# exclude_configs = ['2', '6', 'H->CA->R', 'H->R->CA']
exclude_configs = ['2', '6', 'H->CA->R', 'H->R->CA', '2+2']

make_vs_graph(key_a, key_b, comp_str_a, comp_str_b, exclude=exclude_configs, save_graph_name='fig10b', verticle_lines=1)

In [ ]:
# fig 11a:
key_a = key_b = '11a'

comp_str_a = 'Jacksonville'
comp_str_b = 'Miami'

# ======

dfa =  all_dfs[key_a][0].copy()
dfb =  all_dfs[key_b][1].copy()

# not using these
dfa.drop(dfa.loc[dfa['reconfiguration'].str.fullmatch('H->CA->R')].index, inplace=True)
dfb.drop(dfb.loc[dfb['reconfiguration'].str.fullmatch('H->CA->R')].index, inplace=True)
dfa.drop(dfa.loc[dfa['reconfiguration'].str.fullmatch('H->R->CA')].index, inplace=True)
dfb.drop(dfb.loc[dfb['reconfiguration'].str.fullmatch('H->R->CA')].index, inplace=True)
dfa.drop(dfa.loc[dfa['configuration'].str.fullmatch('2')].index, inplace=True)
dfb.drop(dfb.loc[dfb['configuration'].str.fullmatch('2')].index, inplace=True)
dfa.drop(dfa.loc[dfa['configuration'].str.fullmatch('6')].index, inplace=True)
dfb.drop(dfb.loc[dfb['configuration'].str.fullmatch('6')].index, inplace=True)

# rename cols. mark to remove 2-2 and 6-6 for one side as they are the same and rename the remaining one column with both names
dfa.loc[dfa["configuration"].str.endswith('2+2'), "configuration"] = f"2-2"
dfb.drop(dfb.loc[dfb['configuration'].str.endswith('2+2')].index, inplace=True)
# dfb.loc[dfb["configuration"].str.endswith('2+2'), "configuration"] = "remove"
dfa.loc[dfa["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comp_str_a}"
dfb.loc[dfb["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comp_str_b}"
dfa.loc[dfa["configuration"].str.endswith('6+6'), "configuration"] = f"6-6"
dfb.drop(dfb.loc[dfb['configuration'].str.endswith('6+6')].index, inplace=True)
# dfb.loc[dfb["configuration"].str.endswith('6+6'), "configuration"] = "remove"


vs_df = pd.concat([dfa, dfb])

# remove marked cols 
# vs_df.drop(vs_df.loc[vs_df['configuration'].str.endswith("remove")].index, inplace=True)
# vs_df.drop(vs_df.loc[vs_df['configuration'].str.fullmatch("2-2(remove)")].index, inplace=True)

vs_df['configuration'] = pd.Categorical(vs_df['configuration'], ["2-2", "6-6", f"6+6+6\n{comp_str_a}", f"6+6+6\n{comp_str_b}"]) # sort order
vs_df = vs_df.sort_values("configuration")
vs_df.reset_index(inplace = True)
vs_df.drop(['index'], inplace=True, axis=1)

_make_graph(vs_df, vs_graphs=True, auto_title=False, title='', verticle_lines=0, save_graph_name='fig11a')

In [ ]:
# fig 11b
key_a = key_b = '11b'

comp_str_a = 'Jacksonville'
comp_str_b = 'Miami'

# ======

dfa =  all_dfs[key_a][0].copy()
dfb =  all_dfs[key_b][1].copy()

# not using these
dfa.drop(dfa.loc[dfa['reconfiguration'].str.fullmatch('H->CA->R')].index, inplace=True)
dfb.drop(dfb.loc[dfb['reconfiguration'].str.fullmatch('H->CA->R')].index, inplace=True)
dfa.drop(dfa.loc[dfa['reconfiguration'].str.fullmatch('H->R->CA')].index, inplace=True)
dfb.drop(dfb.loc[dfb['reconfiguration'].str.fullmatch('H->R->CA')].index, inplace=True)
dfa.drop(dfa.loc[dfa['configuration'].str.fullmatch('2')].index, inplace=True)
dfb.drop(dfb.loc[dfb['configuration'].str.fullmatch('2')].index, inplace=True)
dfa.drop(dfa.loc[dfa['configuration'].str.fullmatch('6')].index, inplace=True)
dfb.drop(dfb.loc[dfb['configuration'].str.fullmatch('6')].index, inplace=True)

# rename cols. mark to remove 2-2 and 6-6 for one side as they are the same and rename the remaining one column with both names
dfa.loc[dfa["configuration"].str.endswith('2+2'), "configuration"] = "2-2"
dfb.drop(dfb.loc[dfb['configuration'].str.endswith('2+2')].index, inplace=True)
# dfb.loc[dfb["configuration"].str.endswith('2+2'), "configuration"] = "remove"
dfa.loc[dfa["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comp_str_a}"
dfb.loc[dfb["configuration"].str.endswith('6+6+6'), "configuration"] = f"6+6+6\n{comp_str_b}"
dfa.loc[dfa["configuration"].str.endswith('6+6'), "configuration"] = "6-6"
dfb.drop(dfb.loc[dfb['configuration'].str.endswith('6+6')].index, inplace=True)
# dfb.loc[dfb["configuration"].str.endswith('6+6'), "configuration"] = "remove"


vs_df = pd.concat([dfa, dfb])

# remove marked cols 
# vs_df.drop(vs_df.loc[vs_df['configuration'].str.endswith("remove")].index, inplace=True)
# vs_df.drop(vs_df.loc[vs_df['configuration'].str.fullmatch("2-2(remove)")].index, inplace=True)

vs_df['configuration'] = pd.Categorical(vs_df['configuration'], ["2-2", "6-6", f"6+6+6\n{comp_str_a}", f"6+6+6\n{comp_str_b}"]) # sort order
vs_df = vs_df.sort_values("configuration")
vs_df.reset_index(inplace = True)
vs_df.drop(['index'], inplace=True, axis=1)

_make_graph(vs_df, vs_graphs=True, auto_title=False, title='', verticle_lines=0, save_graph_name='fig11b')


In [ ]:
# fig 11c

key = '11c'
# exclude_configs = ['H->CA->R', '2', '2+2', '6+6+6', '6']
# make_standard_graph(key, exclude=exclude_configs)

df = all_dfs[key][0].copy()

df.drop(df.loc[df['reconfiguration'].str.fullmatch('H->CA->R')].index, inplace=True)
df.drop(df.loc[df['configuration'].str.fullmatch('2')].index, inplace=True)
df.drop(df.loc[df['configuration'].str.fullmatch('6')].index, inplace=True)
df.drop(df.loc[df['configuration'].str.endswith('2+2')].index, inplace=True)
df.drop(df.loc[(df['configuration'].str.endswith('6+6+6') & df['reconfiguration'].str.fullmatch('No'))].index, inplace=True)
df.drop(df.loc[(df['configuration'].str.endswith('6+6') & df['reconfiguration'].str.fullmatch('No'))].index, inplace=True)

_make_graph(df, auto_title=False, title='', relative_width_bar=0.5, fig_w=5, override_x_labels=['rec. 6+6+6\nMiami'], save_graph_name='fig11c')

In [ ]:
# fig 13a:
key = '13a'
exclude_configs = ['2', '2+2', '6']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig13a', overlapping_labels_case=True, override_x_labels=['6-6', '6+6+6', 'rec. 6+6+6', 'rec. 6+6+6\n(preemptive)'])

In [ ]:
# fig 13b:
key = '13b'
exclude_configs = ['2', '2+2', '6']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='fig13b', override_x_labels=['6-6', '6+6+6', 'rec. 6+6+6', 'rec. 6+6+6\n(preemptive)'])

In [ ]:
# fig A1a

key = 'A1a'

exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='figA1a')

In [ ]:
# fig A1b:

key = 'A1b'

exclude_configs = ['H->CA->R', 'H->R->CA']
make_standard_graph(key, exclude=exclude_configs, save_graph_name='figA1b')